In [3]:
from notebook.services.config import ConfigManager
cm = ConfigManager()

#cm.update('livereveal', {'width': 1000, 'height': 600,'scroll': True,})
cm.update('livereveal', {'scroll': True,})

%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.abspath("."))

from viewer import MeshCatViewer
from threejs_viewer import RobotArtist


In [7]:
%%html
<style>
body.rise-enabled div.code_cell{
    font-size:40%;
}
</style>

## Coordinate frames

* World coordinate frame (WCF)

* Robot coordinate frame (RCF)

* Tool0 coordinate frame (T0CF)

* Tool coordinate frame (TCF)

* Object coordinate frame (OCF)

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/02_coordinate_frames.jpg"  />

# Kinematic model

A robotic arm is considered as a kinematic chain of rigid bodies (or links) connected by joints to provide constrained motion.
These links are constrained by their connections to other links.

All rigid body movements are rotations, translations, or combinations of the two.

kinematics = subdomain of mechanics, contrary to dynamics it concerns laws of motion without considering forces

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/01_robot_links_and_joints.jpg" />

In the kinematic model of a robot, the connection of different manipulator joints is known as link, and the 
integration of two or more links is called a joint. This kinematic model can be represented as a tree structure.
The tree describes the kinematic chain, i.e., the connection of robotic links with joints, and the
inter-dependendencies of these links.
This tree structure plus the underlying geometric information can be defined in Unified Robot Description Format (URDF),
which describes any robot. If the robot is mounted on external axes, these links and joints
can be added as well.

## Kinematic chains

<table>
<tr>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/2/2c/Modele_cinematique_corps_humain.svg" style="height: 300px" /></td>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/ATHLETE_robot_climbing_a_hill.jpg/640px-ATHLETE_robot_climbing_a_hill.jpg" style="height: 300px" /></td>
<td><img src="https://upload.wikimedia.org/wikipedia/commons/f/f2/SteamEngine_Boulton%26Watt_1784.png" style="height: 300px" /></td>
</tr>
</table>

### Part of ur5.urdf

```xml
<link name="base_link">
    <visual>
      <geometry>
        <mesh filename="package://ur_description/meshes/ur5/visual/base.obj"/>
      </geometry>
      <material name="LightGrey">
        <color rgba="0.7 0.7 0.7 1.0"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <mesh filename="package://ur_description/meshes/ur5/collision/base.stl"/>
      </geometry>
    </collision>
    <inertial>
      <mass value="4.0"/>
      <origin rpy="0 0 0" xyz="0.0 0.0 0.0"/>
      <inertia ixx="0.00443333156" ixy="0.0" ixz="0.0" iyy="0.00443333156" iyz="0.0" izz="0.0072"/>
    </inertial>
  </link>
  
  <joint name="shoulder_pan_joint" type="revolute">
    <parent link="base_link"/>
    <child link="shoulder_link"/>
    <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.089159"/>
    <axis xyz="0 0 1"/>
    <limit effort="150.0" lower="-6.2831853" upper="6.2831853" velocity="3.15"/>
    <dynamics damping="0.0" friction="0.0"/>
  </joint>
```

### Example UR5 URDF:

https://github.com/ros-industrial/universal_robot/blob/kinetic-devel/ur_description/urdf/ur5.urdf.xacro

In [2]:
from compas.robots import *
import compas_fab
from threejs_viewer import RobotArtist
from threejs_viewer import ThreeJsViewer

r = 'ros-industrial/abb'
p = 'abb_irb6600_support'
b = 'kinetic-devel'

loader = GithubPackageMeshLoader(r, p, b)
urdf = loader.load_urdf('irb6640.urdf')

#loader = LocalPackageMeshLoader(compas_fab.get('universal_robot'), 'ur_description')
#urdf = compas_fab.get('universal_robot/ur_description/urdf/ur5.urdf')

model = RobotModel.from_urdf_file(urdf)
model.load_geometry(loader)
artist = RobotArtist(model)

geo = artist.draw_visual()

viewer = ThreeJsViewer()
viewer.show(geo)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.0), quaterni…

## Joints

The joints are the elements in a robot which helps the links to travel in different kind of movements.
The three major types of joints are:

* Revolute: A hinge joint that rotates along the axis and has a limited range specified by the upper and lower limits.
* Prismatic: A sliding joint that slides along the axis, and has a limited range specified by the upper and lower limits.
* Fixed: Not really a joint because it cannot move, all degrees of freedom are locked.


In [8]:
from compas.robots import Joint

print(Joint.REVOLUTE)
print(Joint.PRISMATIC)
print(Joint.FIXED)

0
2
3


In [9]:
from math import pi
from compas_fab.robots import Configuration

values = [1.5, pi]
types = [Joint.PRISMATIC, Joint.REVOLUTE]

config = Configuration(values, types)
config = Configuration.from_revolute_values([pi/2, 0., 0., pi/2, pi, 0])
config = Configuration.from_prismatic_and_revolute_values([8.312], [pi/2, 0., 0., 0., 2*pi, 0.8])


In [4]:
from compas.robots import RobotModel
from compas.robots import Link
from compas.robots import Joint
from compas.geometry import Frame
from compas.geometry import Transformation
from compas.robots.model.geometry import Origin
from compas.robots.model.geometry import Geometry
from compas.robots.model.link import Visual
from compas.robots.model.geometry import Sphere
from compas.robots.model.joint import Axis
import compas.geometry
from compas.datastructures import Mesh
from compas.geometry import Polyhedron
import math

from compas_fab.artists import BaseRobotArtist

from compas_fab.robots import Configuration

class RobotArtist(BaseRobotArtist):

    def __init__(self, robot):
        super(RobotArtist, self).__init__(robot)

    def transform(self, geometry, transformation):
        geometry.transform(transformation)

    def draw_geometry(self, geometry, color=None):
        return geometry


link0 = Link("world")

joint1 = Joint("sun_joint", 'continuous', "world", "sun_link", origin=Origin.worldXY(), axis=Axis('0 0 1'))
link1 = Link("sun_link", visual=[Visual(Geometry(sphere=Sphere(2.5)))])

earth_origin = Origin((10, 0, 0), (1, 0, 0), (0, 1, 0))
joint2 = Joint("earth_joint", 'continuous', "sun_link", "earth_link", origin=earth_origin, axis=Axis('0 0 1'))
link2 = Link("earth_link", visual=[Visual(Geometry(sphere=Sphere(1)))])

moon_origin = Origin((4, 0, 0), (1, 0, 0), (0, 1, 0))
joint3 = Joint("moon_joint", 'continuous', "earth_link", "moon_link", origin=moon_origin, axis=Axis('0 0 1'))
link3 = Link("moon_link", visual=[Visual(Geometry(sphere=Sphere(0.5)))])

sun_system = RobotModel("sun_system", joints=[joint1, joint2, joint3], links=[link0, link1, link2, link3])

artist = RobotArtist(sun_system)

names = ["sun_joint", "earth_joint", "moon_joint"]
configuration = Configuration([math.radians(30), math.radians(30), 0.0], [Joint.CONTINUOUS] * 3)
artist.update(configuration, names, visual=True, collision=False)


viewer = MeshCatViewer()

for sphere in artist.draw_visual():
    print(viewer.draw_sphere(sphere, color=0xaa00ff))
viewer.jupyter_cell()



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
e4fa8122-f00d-11e9-ac7d-f48c50ed471a
e4fc087e-f00d-11e9-817e-f48c50ed471a
e4fca49e-f00d-11e9-b0a8-f48c50ed471a


In [ ]:
from compas.geometry import *

s = Sphere(Point(5.,0,0,), 4)
T = Transformation.from_matrix([[    0.8776,   -0.4794,    0.0000,    1.8363],
 [    0.4794,    0.8776,    0.0000,   -7.1914],
 [    0.0000,    0.0000,    1.0000,    0.0000],
 [    0.0000,    0.0000,    0.0000,    1.0000]])
s.transform(T)
print(s)
p = Point(5.000, -0.000, 0.000)
p.transform(T)
print(p)
transform_points([[5.000, -0.000, 0.000]], T.matrix)[0]


In [ ]:
#from compas_fab.robots import Configuration

"""
viewer = ThreeJsViewer()
names = model.get_configurable_joint_names()
config = Configuration.from_revolute_values([0, -pi/2, pi/2, -pi/2, -pi/2, 0])
artist.update(config, names)
geo = artist.draw_visual()
viewer.show(geo)
"""

In [ ]:
import compas
from compas.geometry import Frame
from compas.geometry import Box
from compas.datastructures import Mesh
from compas.datastructures import mesh_quads_to_triangles

import pythreejs as p3js
from IPython.display import display

frame = Frame([1, 0, 0], [-0.45, 0.1, 0.3], [1, 0, 0])
width, length, height = 10, 10, 10

box = Box(frame, width, length, height)
mesh = Mesh.from_vertices_and_faces(box.vertices, box.faces)
mesh_quads_to_triangles(mesh)

vertices, faces = mesh.to_vertices_and_faces()

vertexcolors = ['#777777'] * len(vertices)
faces = [f + [None, [vertexcolors[i] for i in f], None] for f in faces]

geom = p3js.Geometry(vertices=vertices, faces=faces, colors=vertexcolors)
geom.exec_three_obj_method('computeFaceNormals')

m3 = p3js.Mesh(geometry=geom,
          material=p3js.MeshLambertMaterial(vertexColors='VertexColors'),
          position=[-0.5, -0.5, -0.5])

cCube = p3js.PerspectiveCamera(position=[10, 10, 10], fov=20,
                      children=[p3js.DirectionalLight(color='#ffffff', position=[-3, 5, 1], intensity=0.5)])
sceneCube = p3js.Scene(children=[m3, cCube, p3js.AmbientLight(color='#dddddd')])


rendererCube = p3js.Renderer(camera=cCube, background='black', background_opacity=1,
                        scene=sceneCube, controls=[p3js.OrbitControls(controlling=cCube)])

display(rendererCube)

In [ ]:
import time
from roslibpy import Topic
from compas_fab.backends import RosClient

client = RosClient()
client.run()
topic = Topic(client, '/messages', 'std_msgs/String')
topic.advertise()

while True:
    topic.publish(dict(data='Hello world'))
    time.sleep(1)

client.terminate()

## ROS Protocol: Message types

* Dictionary-like data structure
* Defines the type of topics
* Large collection of predefined types

In [ ]:
from compas_fab.backends.ros import String

str_msg = String('hello world')
str_msg.msg

In [ ]:
from compas.geometry import Frame
from compas_fab.backends.ros import Pose

frame = Frame.worldXY()
pose = Pose.from_frame(frame)
pose.msg

# Forward and inverse kinematics

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/02_forward_and_inverse_kinematics.jpg" />